# **네이버 영화 리뷰 - 예측하기**

구글 드라이브 마운트

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


Mecab(형태소 분석기) 설치

In [2]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 91 (delta 43), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% (91/91), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
     |████████████████████████████████| 19.4 MB 6.1 MB/s 
     |████████████████████████████████| 448 kB 52.0 MB/s 
     |████████████████████████████████| 86 kB 5.8 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2021-09-26 12:06:40--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00

예측 함수

In [3]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer # 토큰화
from tensorflow.keras.preprocessing.sequence import pad_sequences # 패딩
from konlpy.tag import Mecab # 형태소 분석

# 정수 인코딩 사전 처리
x_train_tokenized = np.load("/content/drive/MyDrive/ml_data/NaverMovieReview/x_train_tokenized.npy", allow_pickle=True)

mecab = Mecab()

vocab_size = 28631
tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 
tokenizer.fit_on_texts(x_train_tokenized)

# 불용어
stopwords = ["도", "는", "다", "의", "가", "이", "은", "한", "에", "하", "고", "을", "를", "인", "듯", "과", "와", "네", "들", "듯", "지", "임", "게"]
loaded_model = load_model("/content/drive/MyDrive/ml_data/models/NaverMovieReview/best_model_GRU.h5")
#loaded_model = load_model("/content/drive/MyDrive/ml_data/models/NaverMovieReview/best_model_LSTM.h5")

max_len = 50

def predict(text):
    tokenized = mecab.morphs(text) # 입력한 text 토큰화
    tokenized = [word for word in tokenized if not word in stopwords] # 토근화한 text list 중 불용어 제거
    encoded = tokenizer.texts_to_sequences([tokenized]) # 정수 인코딩
    padded = pad_sequences(encoded, maxlen = max_len) # 패딩
    score = float(loaded_model.predict(padded)) # 예측
    if score > 0.5:
        print("{:.2f}% 확률로 긍정 리뷰입니다.".format(score * 100))
    else:
        print("{:.2f}% 확률로 부정 리뷰입니다.".format((1 - score) * 100))

In [4]:
predict('''
지브리 무드가 잘 묻어나서 어느 포인트인지는 잘 모르겠지만 
소소하게 위로받고 스트레스 풀리는 지브리만의 분위기가 있는것 같아요 아직까지 생각난답니다
''')

99.96% 확률로 긍정 리뷰입니다.


In [6]:
predict('''
박정민, 이성민 등 배우들만 믿고 아무 생각없이 영화관에 들어 섰다가...
펑펑 울고 나왔습니다.
저는 딸을 둘 둔 아빠입니다.
가족이 있는 모든 분들께 진심을 담아 추천합니다.
가족과 함께 보시면 더 좋습니다.
''')

99.84% 확률로 긍정 리뷰입니다.


In [7]:
predict('''
마음 따뜻한 이들의 서로를 향한 가슴 찡한 이야기.
어수선한 시국에 많은 사람들에게 위로가 되길 바랍니다.
''')

99.99% 확률로 긍정 리뷰입니다.


In [8]:
predict('''
너무 억지 감동 끌어내는 듯 하네요  평점 기대보다 이하일 겁니다.
''')

99.97% 확률로 부정 리뷰입니다.


In [9]:
predict('''
빌리 엘리엇을 중요할때 넘 마니 카피해서 잘 보다가 황당했고 
마무리에 넘 유명한 ost릉 사용해서 안타깝더만~~
''')

99.98% 확률로 부정 리뷰입니다.
